<a href="https://colab.research.google.com/github/ramizcihe/week3-cihe240058/blob/main/WEEK3_WEEKLY_REPORT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Install if not already available
!pip install pandas numpy nltk textblob contractions scikit-learn emoji

# Imports
import pandas as pd
import numpy as np
import re
import string
import nltk
from textblob import TextBlob
import contractions
import emoji
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
# I have dataset in github, I copied that url and Load it in Colab with pandas
import pandas as pd

# Replace this with your own raw GitHub link
url = "https://raw.githubusercontent.com/ramizcihe/week3-cihe240058/refs/heads/main/cyberbullying_tweets.csv"

# Read the CSV file directly
df = pd.read_csv(url)

# Check first few rows
df.head()


,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [27]:
# Lets read the dataset
import pandas as pd

# Read the dataset
df = pd.read_csv("cyberbullying_tweets.csv")

# Check first few rows and column names
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [28]:
#Handling Missing or Blank Entries
# Remove missing values in text column
df = df.dropna(subset=["tweet_text"])

# Remove completely blank posts
df = df[df["tweet_text"].str.strip() != ""]

df.reset_index(drop=True, inplace=True)
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [29]:
#Remove Duplicates & Irrelevant Content
# Remove duplicate tweets
df = df.drop_duplicates(subset=["tweet_text"])

# Filter irrelevant tweets (only hashtags, ads, or spam links)
import re

def is_irrelevant(text):
    if re.fullmatch(r'(#\w+\s*)+', text):
        return True
    if "http" in text.lower() or "www" in text.lower():
        return True
    return False

df = df[~df["tweet_text"].apply(is_irrelevant)]
df.reset_index(drop=True, inplace=True)
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [30]:
#Clean and Normalize Text Data
import nltk
import string
import contractions
import emoji
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(nltk.corpus.stopwords.words("english"))
lemmatizer = nltk.WordNetLemmatizer()

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Expand contractions
    text = contractions.fix(text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove emojis
    text = emoji.replace_emoji(text, replace="")
    # Remove special characters & digits
    text = re.sub(r"[^a-z\s]", "", text)
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["tweet_text"].apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,tweet_text,cyberbullying_type,clean_text
0,"In other words #katandandre, your food was cra...",not_cyberbullying,in other words katandandre your food was crapi...
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,why is aussietv so white mkr theblock imaceleb...
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,xochitlsuckkks a classy whore or more red velv...
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,jasongio meh p thanks for the heads up but not...
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,rudhoeenglish this is an isis account pretendi...


In [31]:
# Install NLTK
import nltk

# Download the required resources
nltk.download('punkt')
nltk.download('punkt_tab')  # This is the missing one
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [32]:
#Tokenize and Lemmatize Text
def tokenize_lemmatize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

df["tokens"] = df["clean_text"].apply(tokenize_lemmatize)
df.head()

,tweet_text,cyberbullying_type,clean_text,tokens
0,"In other words #katandandre, your food was cra...",not_cyberbullying,in other words katandandre your food was crapi...,"[word, katandandre, food, crapilicious, mkr]"
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,why is aussietv so white mkr theblock imaceleb...,"[aussietv, white, mkr, theblock, imacelebritya..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,xochitlsuckkks a classy whore or more red velv...,"[xochitlsuckkks, classy, whore, red, velvet, c..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,jasongio meh p thanks for the heads up but not...,"[jasongio, meh, p, thanks, head, concerned, an..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,rudhoeenglish this is an isis account pretendi...,"[rudhoeenglish, isi, account, pretending, kurd..."


In [33]:
#Filter Stop Words (Optional Step)
df["no_stopwords"] = df["tokens"].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

,tweet_text,cyberbullying_type,clean_text,tokens,no_stopwords
0,"In other words #katandandre, your food was cra...",not_cyberbullying,in other words katandandre your food was crapi...,"[word, katandandre, food, crapilicious, mkr]","[word, katandandre, food, crapilicious, mkr]"
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,why is aussietv so white mkr theblock imaceleb...,"[aussietv, white, mkr, theblock, imacelebritya...","[aussietv, white, mkr, theblock, imacelebritya..."
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,xochitlsuckkks a classy whore or more red velv...,"[xochitlsuckkks, classy, whore, red, velvet, c...","[xochitlsuckkks, classy, whore, red, velvet, c..."
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,jasongio meh p thanks for the heads up but not...,"[jasongio, meh, p, thanks, head, concerned, an...","[jasongio, meh, p, thanks, head, concerned, an..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,rudhoeenglish this is an isis account pretendi...,"[rudhoeenglish, isi, account, pretending, kurd...","[rudhoeenglish, isi, account, pretending, kurd..."


In [34]:
#Encode Target Variable and Categorical Features
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["cyberbullying_type"])

# If you have other categorical metadata like 'platform', encode them too
# platform_encoder = LabelEncoder()
# df["platform_encoded"] = platform_encoder.fit_transform(df["platform"])

df.head()

,tweet_text,cyberbullying_type,clean_text,tokens,no_stopwords,label_encoded
0,"In other words #katandandre, your food was cra...",not_cyberbullying,in other words katandandre your food was crapi...,"[word, katandandre, food, crapilicious, mkr]","[word, katandandre, food, crapilicious, mkr]",3
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,why is aussietv so white mkr theblock imaceleb...,"[aussietv, white, mkr, theblock, imacelebritya...","[aussietv, white, mkr, theblock, imacelebritya...",3
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,xochitlsuckkks a classy whore or more red velv...,"[xochitlsuckkks, classy, whore, red, velvet, c...","[xochitlsuckkks, classy, whore, red, velvet, c...",3
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,jasongio meh p thanks for the heads up but not...,"[jasongio, meh, p, thanks, head, concerned, an...","[jasongio, meh, p, thanks, head, concerned, an...",3
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,rudhoeenglish this is an isis account pretendi...,"[rudhoeenglish, isi, account, pretending, kurd...","[rudhoeenglish, isi, account, pretending, kurd...",3


In [35]:
#Perform Feature Engineering
from textblob import TextBlob

# Word and character counts
df["word_count"] = df["clean_text"].apply(lambda x: len(x.split()))
df["char_count"] = df["clean_text"].apply(len)

# Sentiment score
df["sentiment"] = df["clean_text"].apply(lambda x: TextBlob(x).sentiment.polarity)

# Presence of abusive keywords
abusive_words = {"hate", "stupid", "loser", "idiot", "dumb"}
df["abusive_flag"] = df["clean_text"].apply(lambda x: any(word in x.split() for word in abusive_words))

df.head()


,tweet_text,cyberbullying_type,clean_text,tokens,no_stopwords,label_encoded,word_count,char_count,sentiment,abusive_flag
0,"In other words #katandandre, your food was cra...",not_cyberbullying,in other words katandandre your food was crapi...,"[word, katandandre, food, crapilicious, mkr]","[word, katandandre, food, crapilicious, mkr]",3,9,57,-0.125,False
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,why is aussietv so white mkr theblock imaceleb...,"[aussietv, white, mkr, theblock, imacelebritya...","[aussietv, white, mkr, theblock, imacelebritya...",3,14,102,0.000,False
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,xochitlsuckkks a classy whore or more red velv...,"[xochitlsuckkks, classy, whore, red, velvet, c...","[xochitlsuckkks, classy, whore, red, velvet, c...",3,9,57,0.200,False
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,jasongio meh p thanks for the heads up but not...,"[jasongio, meh, p, thanks, head, concerned, an...","[jasongio, meh, p, thanks, head, concerned, an...",3,18,96,-0.150,False
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,rudhoeenglish this is an isis account pretendi...,"[rudhoeenglish, isi, account, pretending, kurd...","[rudhoeenglish, isi, account, pretending, kurd...",3,18,98,0.000,False


In [36]:
# Vectorize Text Data
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df["clean_text"])
X_tfidf.shape

(41760, 56369)

In [37]:
#Split Dataset
from sklearn.model_selection import train_test_split

X = X_tfidf
y = df["label_encoded"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

len(y_train), len(y_val), len(y_test)


(29232, 6264, 6264)

In [38]:
summary = """
At this point, I have:
✔ Cleaned text
✔ Tokenized + lemmatized words
✔ Removed stopwords
✔ Encoded target labels
✔ Created extra features (sentiment, abusive word flag, length)
✔ Vectorized text into TF-IDF
✔ Split into train, validation, and test sets
"""

print(summary)



At this point, I have:
✔ Cleaned text
✔ Tokenized + lemmatized words
✔ Removed stopwords
✔ Encoded target labels
✔ Created extra features (sentiment, abusive word flag, length)
✔ Vectorized text into TF-IDF
✔ Split into train, validation, and test sets



In [40]:
from IPython.display import HTML

HTML("""
<div style="background-color:#f0f8ff; border: 2px solid #4682b4; padding: 15px; border-radius: 10px; width: 50%; font-family: Arial; color: #2f4f4f;">
    <h2 style="color:#4682b4; text-align:center;">Student Information</h2>
    <p><b>Name:</b> <span style="color:#ff4500;">Ramiz Ahmed</span></p>
    <p><b>Student ID:</b> <span style="color:#228b22;">CIHE240058</span></p>
    <p><b>Professor:</b> <span style="color:#8a2be2;">Dr. Md Ashraf Uddin</span></p>
</div>
""")
